In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install monai
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 28.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.0 MB 21.1 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3


In [ ]:
# !pip install torchvision
# !pip install torch
# !pip install matplotlib
# !pip install monai
# !pip install nibabel
# !pip install -U scikit-learn scipy matplotlib
# !pip install -U numpy 
# !pip install tensorboard

In [1]:
import sys
import os
py_file_location = "/content/drive/MyDrive/liver-imaging-analysis/engine"
sys.path.append(os.path.abspath(py_file_location))


In [1]:
import torch
import nibabel
import engine
import models
import json
from torch import nn
import losses


#Read configurations from JSON File
with open('../config/configs.json') as f:
    config = json.load(f)

%load_ext tensorboard
# %tensorboard --logdir "/content/drive/MyDrive/liver-imaging-analysis/engine/runs/"

#Instance from the Model
model= engine.Engine(
    config=config,
    device="cuda" if torch.cuda.is_available() else "cpu",
    metrics=["loss"],
    training_data_path=config['dataset']['training'],
    testing_data_path=config['dataset']['testing'],
    transformation_flag=True,
    data_size=config['transforms']['transformation_size'],
    batchsize=config["training"]['batch_size'],
    train_valid_split=0,
)

#Print loss function, optimizer, and metrics
print("Compile Status")
print("-------------------------------")
model.compile_status()

#Print Dataset Shape
print("\nData Status")
print("-------------------------------")
model.data_status()

# Load Stored Checkpoint
model.load_checkpoint(config['potential_checkpoint'])

# Start Fitting
print("\nTraining")
print("-------------------------------")
model.fit(
    epochs=config["training"]['epochs'],
    evaluation_set=model.test_dataloader,
    evaluate_epochs=1,
    visualize_epochs=1,
    save_flag=True,
    save_path=config["potential_checkpoint"]
)

# #Test model on testing set
print("\nTesting")
print("-------------------------------")
print("Training loss=",model.test(model.train_dataloader))
print("Testing loss=",model.test(model.test_dataloader))

#Predict mask for a single volume
# print("\nPrediction")
# print("-------------------------------")
# ImagePath=config['ImagePath']
# output=model.predict(ImagePath)
# output.shape


Used Device:  cuda
Compile Status
-------------------------------
Loss= DiceLoss(
  (normalization): Sigmoid()
) 

Optimizer= Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.01
    maximize: False
    weight_decay: 0
) 

Metrics= ['loss'] 


Data Status
-------------------------------
Batch Shape of Training Features: (16, 1, 128, 128) torch.float32
Batch Shape of Training Labels: (16, 1, 128, 128) torch.float32

Testing
-------------------------------
Training loss= 0.058708906173706055
Testing loss= 0.058708906173706055


In [ ]:
# #if model is better, update checkpoint:
# model.save_checkpoint(config["model_checkpoint"])